In [ ]:
import torch
import torchvision
import torch.nn as nn
import torch.nn.functional as F
import torchvision.transforms as transforms
from __future__ import print_function, division
import os
import pandas as pd
from skimage import io, transform
import numpy as np
import matplotlib.pyplot as plt
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms, utils

# Ignore warnings
import warnings
warnings.filterwarnings("ignore")

plt.ion()   # interactive mode



In [ ]:
trainer_names_csv = pd.read_csv('/Users/devpatelio/Downloads/Coding/Python/pyTorch/diabetic-retinopathy-224x224-gaussian-filtered/train.csv')
n = len(trainer_names_csv)
# data_labels = pd.DataFrame(data = trainer_names_csv)
img_names = trainer_names_csv.iloc[:n, 0]
label = trainer_names_csv.iloc[:n, 1]

In [ ]:
from torch.utils.data import Dataset
import pandas as pd
import os
from PIL import Image
import torch

class DRDataset(Dataset):
    def __init__ (self, csv_file, root_dir, transform=None):
        """
        csv_file = labels
        root_dir = images
        
        """
        
        self.train_data_csv = pd.read_csv(csv_file)
        self.root_dir = root_dir
        self.transform = transform
        
    def __len__(self):
        return len(self.train_data_csv)

    def __getitem__ (self, idx): 
        name = self.train_data_csv.iloc[idx, 0]
        img_name = os.path.join(self.root_dir, name +'.png') #stores image id 
        image = Image.open(img_name).convert("RGB") #return image based on the name of the image
        y_labels = torch.tensor(float(self.train_data_csv.iloc[idx, 1:])) #converts the labels in the csv to floats in tensor format

        
        if self.transform is not None:
            image = self.transform(image)
        return (image, y_labels) #returns the image in tensor format along with its corresponding labels

In [ ]:
from tqdm import tqdm
import random

transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225]),
   
])

num_epochs = 10
learning_rate = 0.01
train_CNN = True
batch_size = 10
shuffle = True
pin_memory = True
num_workers = 0



train_size = int(0.7 * n)
validation_size = n - train_size


image_dataset = DRDataset(csv_file='/Users/devpatelio/Downloads/Coding/Python/pyTorch/diabetic-retinopathy-224x224-gaussian-filtered/train.csv', root_dir='/Users/devpatelio/Downloads/Coding/Python/pyTorch/diabetic-retinopathy-224x224-gaussian-filtered/gaussian_filtered_images/gaussian_filtered_images/')

dataset = DRDataset(csv_file='/Users/devpatelio/Downloads/Coding/Python/pyTorch/diabetic-retinopathy-224x224-gaussian-filtered/train.csv', root_dir='/Users/devpatelio/Downloads/Coding/Python/pyTorch/diabetic-retinopathy-224x224-gaussian-filtered/gaussian_filtered_images/gaussian_filtered_images/', 
                               transform=transform)

train_set, validation_set = torch.utils.data.random_split(dataset, [train_size, validation_size])

train_loader = torch.utils.data.DataLoader(dataset=train_set, shuffle=shuffle, batch_size=batch_size,num_workers=num_workers, pin_memory=pin_memory)
validation_loader = torch.utils.data.DataLoader(dataset=validation_set, shuffle=shuffle, batch_size=batch_size,num_workers=num_workers, pin_memory=pin_memory)




dataiter = iter(train_loader)
image, label = dataiter.next()

# print(data[0].shape)
# print(data[1].shape)
# print(data[1])
print(image.shape)
print(label.shape)
print(label)
print(image)

for i in range(len(image_dataset)): #indexing through all the samples in the dataset (3662)
    sample = image_dataset[i] #indexes an item in the image dataset
    ax = plt.subplot(1, 4, i+1) #1 row, 4 images per row
    plt.tight_layout()
    ax.set_title('Sample #{}'.format(i))
    plt.imshow(sample[0])
    ax.axis('off')
    print(image_dataset[i])
    
    if i==3: #stops the program at 3 images but you can change this
        break

In [ ]:
from __future__ import print_function, division

import torch
import torch.nn as nn
import torch.optim as optim
from torch.optim import lr_scheduler
import numpy as np
import torchvision
from torchvision import datasets, models, transforms
import time
import os
import copy
import cv2

plt.ion()

data_transforms = {
    'train': transforms.Compose([
        transforms.RandomResizedCrop(224),
        transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
    'val': transforms.Compose([
        transforms.Resize(256),
        transforms.CenterCrop(224),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
}


train_set, validation_set = torch.utils.data.random_split(dataset, [train_size, validation_size])
image_dataset = {'train': train_set, 'val': validation_set}
dataloaders = {x: torch.utils.data.DataLoader(image_dataset[x], batch_size=batch_size, shuffle=True) for x in ['train', 'val']} 
dataset_sizes = {x: len(image_dataset[x]) for x in ['train', 'val']}

device = torch.device("cuda: 0" if torch.cuda.is_available() else "cpu")

def imshow(inp, title=None):
    inp = inp.numpy().transpose((1, 2, 0))
    mean = np.array([0.485, 0.456, 0.406])
    std = np.array([0.229, 0.224, 0.225])
    inp = std * inp + mean
    inp = np.clip(inp, 0, 1)
    plt.imshow(inp)
    if title is not None: 
        plt.title(title)
    plt.pause(0.001)

dataiters = iter(dataloaders['train'])
images, labels = dataiters.next()

out = torchvision.utils.make_grid(inputs, 5)
imshow(out, title=f'{labels}')

def train_model(model, criterion, optimizer, scheduler, num_epochs):
    since = time.time()
    best_models_wts = copy.deepcopy(model.state_dict())
    best_acc = 0.0
    
    for epoch in range(num_epochs):
        print(f'Epoch {epoch}/{num_epochs-1}')
        print('-' * 10)
        
        for phase in ['train', 'val']:
            if phase == 'train':
                model.train() 
            else: 
                model.eval()
        
            running_loss = 0.0
            running_corrects = 0

            for inputs, labels in dataloaders[phase]:
                inputs = inputs.to(device)
                labels = labels.to(device)

                optimizer.zero_grad()

                with torch.set_grad_enabled(phase == 'train'):
                    outputs = model(inputs)
                    _, preds = torch.max(outputs, 1)
                    loss = criterion(outputs, labels.type(torch.LongTensor))
                    if phase == 'train':
                        loss.backward()
                        optimizer.step()

                running_loss =+ loss.item() * inputs.size(0)
                running_corrects += torch.sum(preds == labels.data)
            if phase == 'train':
                scheduler.step()

            epoch_loss = running_loss/dataset_sizes[phase]
            epoch_acc = running_corrects.double()/dataset_sizes[phase]

            print('{} Loss: {:.4f} Acc: {:.4f}'.format(phase, epoch_loss, epoch_acc))

            if phase == 'val' and epoch_acc > best_acc:
                best_acc = epoch_acc
                best_model_wts = copy.deepcopy(model.state_dict())

        print()

    time_elapsed = time.time() - since
    print('Training complete in {:.0f}m {:.0f}s'.format(time_elapsed//60, time_elapsed % 60))
    print('Best val Acc: {:4f}'.format(best_acc))

    model.load_state_dict(best_model_wts)
    return model

In [ ]:
def visualize_model(model, num_images=6): 
    was_training = mode.training
    model.eval()
    images_so_far = 0
    fig = plt.figure()
    
    with torch.no_grad():
        for i, (inputs, labels) in enumerate(dataloaders['val']):
            inputs = inputs.to(device)
            labels = labels.to(device)
            
            outputs = model(inputs)
            _, preds = torch.max(outputs, 1)
            
            for j in range(inputs.size()[0]):
                images_so_far += 1
                ax = plt.subplot(num_images//2, 2, images_so_far)
                ax.axis('off')
                ax.set_title('predicted: {}'.format(class_names[preds[j]]))
                imshow(inputs.cpu().data[j])

                if images_so_far == num_images:
                    model.train(mode=was_training)
                    return
        model.train(mode=was_training)

In [ ]:
model_ft = models.resnet18(pretrained=True)
num_ftrs = model_ft.fc.in_features
# Here the size of each output sample is set to 2.
# Alternatively, it can be generalized to nn.Linear(num_ftrs, len(class_names)).
model_ft.fc = nn.Linear(num_ftrs, 5)

model_ft = model_ft.to(device)

criterion = nn.CrossEntropyLoss()

# Observe that all parameters are being optimized
optimizer_ft = optim.SGD(model_ft.parameters(), lr=0.001, momentum=0.9)

# Decay LR by a factor of 0.1 every 7 epochs
exp_lr_scheduler = lr_scheduler.StepLR(optimizer_ft, step_size=7, gamma=0.1)

model_ft = train_model(model_ft, criterion, optimizer_ft, exp_lr_scheduler,
                       num_epochs=25)


In [ ]:
class Net(nn.Module):
    def __init__ (self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(3, 8, 5) ## nn.Conv2d(input channels (image), output channels (no. of kernels), kernel size)
        self.pool = nn.MaxPool2d(2, 2)
        self.conv2 = nn.Conv2d(8, 8, 5)
        self.pool = nn.MaxPool2d(2, 2)
        self.conv3 = nn.Conv2d(8, 20, 5)
        self.pool = nn.MaxPool2d(2, 2)

        self.fc1 = nn.Linear(11520, 2000)
        self.fc2 = nn.Linear(2000, 120)
        self.fc3 = nn.Linear(120, 80)
        self.fc4 = nn.Linear(80, 5)
        
        self.dropout = nn.Dropout(0.2)

        
    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = self.pool(F.relu(self.conv3(x)))
        x = self.dropout(x)

#         print(x.shape)
        x = x.view(-1, 20*24*24)
        x = F.relu(self.fc1(x))
        x = self.dropout(x)
        x = F.relu(self.fc2(x))
        x = F.relu(self.fc3(x))
        x = self.dropout(x)
        x = self.fc4(x)
        return x
    
net = Net()

# sequential = nn.Sequential(
#       nn.Conv2d(in_channels=3, out_channels=16, kernel_size=3)
#     , nn.ReLU()
#     , nn.MaxPool2d(kernel_size=2, stride=2)
#     , nn.Dropout(0.2)
#     , nn.Conv2d(in_channels=16, out_channels=64, kernel_size=3)
#     , nn.ReLU()
#     , nn.MaxPool2d(kernel_size=2, stride=2)
#     , nn.Dropout(0.2)
#     , nn.Conv2d(in_channels=64, out_channels=128, kernel_size=3)
#     , nn.ReLU()
#     , nn.MaxPool2d(kernel_size=2, stride=2)
#     , nn.Dropout(0.2)
#     , nn.Flatten(start_dim=1)  
#     , nn.Linear(in_features=86528, out_features=512)
#     , nn.ReLU()
#     , nn.Dropout(0.2)
#     , nn.Linear(in_features=512, out_features=5)
# )

In [ ]:
import torch.optim as optim

criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(net.parameters(), lr=learning_rate, momentum=0.9)

In [ ]:
for epoch in range(num_epochs):
    running_loss = 0.0
    for i, data in enumerate(train_loader, 0):
        inputs, labels = data
        labels = labels.type(torch.LongTensor)
        optimizer.zero_grad()
        outputs = net(inputs)
#         print(labels.data)
#         print(outputs.data)
        loss = criterion(outputs, labels)
        running_loss += loss.item()
        if i % 10 == 9:    # print every 2000 mini-batches
            print('[%d, %5d] loss: %.3f' %
                  (epoch + 1, i + 1, running_loss / 10))
            running_loss = 0.0
        


print('Finished Training')

In [ ]:
correct = 0
total = 0
with torch.no_grad():
    for data in validation_loader:
        images, labels = data
        outputs = sequential(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

print(f'Accuracy of the network on the {len(validation_loader)} test images: %d %%' % (
    100 * correct / total))

In [ ]:
activation = {}
def get_activation(name):
    def hook(model, input, output):
        activation[name] = output.detach()
    return hook

model.conv1.register_forward_hook(get_activation('conv1'))
data, _ = dataset[0]
data.unsqueeze_(0)
output = model(data)

act = activation['conv1'].squeeze()
fig, axarr = plt.subplots(act.size(0))
for idx in range(act.size(0)):
    axarr[idx].imshow(act[idx])